In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_shuma_shouji=pd.DataFrame(columns=columns)
channel_shuma_shouji

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[8]/div/a/span').click()  #选择频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[8]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'数码'

In [9]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').click()  #选择子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').text
sub_channel

'手机平板'

In [10]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [11]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_shuma_shouji.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 
    


2020-12-16 12:03:49.641327


<ipython-input-11-cb972e047cdf>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-cb972e047cdf>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-cb972e047cdf>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1w7411k7Ay视频属性的抓取
BV号为1w7411k7Ay的视频属性抓取结束


<ipython-input-11-cb972e047cdf>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1f7411r7Pd视频属性的抓取
BV号为1f7411r7Pd的视频属性抓取结束
开始BV号为1z7411B7w4视频属性的抓取
BV号为1z7411B7w4的视频属性抓取结束
开始BV号为157411e7XB视频属性的抓取
BV号为157411e7XB的视频属性抓取结束
开始BV号为147411z7sm视频属性的抓取
BV号为147411z7sm的视频属性抓取结束
开始BV号为1tJ411V77q视频属性的抓取
BV号为1tJ411V77q的视频属性抓取结束
开始BV号为1yJ411n7nA视频属性的抓取
BV号为1yJ411n7nA的视频属性抓取结束
开始BV号为14J411L7uC视频属性的抓取
BV号为14J411L7uC的视频属性抓取结束
开始BV号为1S7411Y7mN视频属性的抓取
BV号为1S7411Y7mN的视频属性抓取结束
开始BV号为1YJ41157sb视频属性的抓取
BV号为1YJ41157sb的视频属性抓取结束
开始BV号为1t7411a7QN视频属性的抓取
BV号为1t7411a7QN的视频属性抓取结束
开始BV号为1m7411k7dg视频属性的抓取
BV号为1m7411k7dg的视频属性抓取结束
开始BV号为14J411n7Nk视频属性的抓取
BV号为14J411n7Nk的视频属性抓取结束
开始BV号为1o741167PS视频属性的抓取
BV号为1o741167PS的视频属性抓取结束
开始BV号为1SJ411n7i4视频属性的抓取
BV号为1SJ411n7i4的视频属性抓取结束
开始BV号为1mJ411E7v2视频属性的抓取
BV号为1mJ411E7v2的视频属性抓取结束
开始BV号为1y7411e7nV视频属性的抓取
BV号为1y7411e7nV的视频属性抓取结束
开始BV号为1ZJ41177YU视频属性的抓取
BV号为1ZJ41177YU的视频属性抓取结束
开始BV号为1BJ411H7H8视频属性的抓取
BV号为1BJ411H7H8的视频属性抓取结束
开始BV号为1oJ411j7d2视频属性的抓取
BV号为1oJ411j7d2的视频属性抓取结束
开始BV号为16J41157es视频属性的抓取
BV号为16J41157es的视频属性抓取结束
开始BV号为1tJ411G75w视频属性的抓取
BV号为1tJ411G75w的视

...

BV号为1zQ4y1A7y6的视频属性抓取结束
开始BV号为1Qa4y1v7eL视频属性的抓取
BV号为1Qa4y1v7eL的视频属性抓取结束
开始BV号为1qe411p7PE视频属性的抓取
BV号为1qe411p7PE的视频属性抓取结束
开始BV号为1cz411z78L视频属性的抓取
BV号为1cz411z78L的视频属性抓取结束
开始BV号为1wV411C7Ch视频属性的抓取
BV号为1wV411C7Ch的视频属性抓取结束
开始BV号为1uV411d7pu视频属性的抓取
BV号为1uV411d7pu的视频属性抓取结束
开始BV号为1iv411z74Y视频属性的抓取
BV号为1iv411z74Y的视频属性抓取结束
开始BV号为1Jg4y167nA视频属性的抓取
BV号为1Jg4y167nA的视频属性抓取结束
开始BV号为1XZ4y1s7w9视频属性的抓取
BV号为1XZ4y1s7w9的视频属性抓取结束
开始BV号为1CV411d7rq视频属性的抓取
BV号为1CV411d7rq的视频属性抓取结束
开始BV号为1zt4y1C7S1视频属性的抓取
BV号为1zt4y1C7S1的视频属性抓取结束
开始BV号为1k54y1D72A视频属性的抓取
BV号为1k54y1D72A的视频属性抓取结束
开始BV号为1QT4y1u7aD视频属性的抓取
BV号为1QT4y1u7aD的视频属性抓取结束
开始BV号为1Zf4y1m7rm视频属性的抓取
BV号为1Zf4y1m7rm的视频属性抓取结束
开始BV号为1Pz4y197gL视频属性的抓取
BV号为1Pz4y197gL的视频属性抓取结束
开始BV号为1dz411q7xi视频属性的抓取
BV号为1dz411q7xi的视频属性抓取结束
开始BV号为16z411v7p4视频属性的抓取
BV号为16z411v7p4的视频属性抓取结束
开始BV号为1Da4y1i7wC视频属性的抓取
BV号为1Da4y1i7wC的视频属性抓取结束
开始BV号为1ep4y197iE视频属性的抓取
BV号为1ep4y197iE的视频属性抓取结束
开始BV号为1az4y1d72D视频属性的抓取
BV号为1az4y1d72D的视频属性抓取结束
开始BV号为1Rf4y117hE视频属性的抓取
BV号为1Rf4y117hE的视频属性抓取结束
开始BV号为14p4y1Q7bs

<ipython-input-11-cb972e047cdf>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1bT4y1J7MS视频属性的抓取
BV号为1bT4y1J7MS的视频属性抓取结束
开始BV号为1Ea4y1v7Ty视频属性的抓取
BV号为1Ea4y1v7Ty的视频属性抓取结束
开始BV号为1eZ4y1W7Pr视频属性的抓取
BV号为1eZ4y1W7Pr的视频属性抓取结束
开始BV号为1d54y1B7dW视频属性的抓取
BV号为1d54y1B7dW的视频属性抓取结束
开始BV号为19f4y1271z视频属性的抓取
BV号为19f4y1271z的视频属性抓取结束
开始BV号为1UA411v7Xp视频属性的抓取
BV号为1UA411v7Xp的视频属性抓取结束
开始BV号为1K54y1z7LU视频属性的抓取
BV号为1K54y1z7LU的视频属性抓取结束
开始BV号为1NK411p7F8视频属性的抓取
BV号为1NK411p7F8的视频属性抓取结束
开始BV号为15K411H7eP视频属性的抓取
BV号为15K411H7eP的视频属性抓取结束
开始BV号为1Ez411i7bS视频属性的抓取
BV号为1Ez411i7bS的视频属性抓取结束
开始BV号为1jK411p7sa视频属性的抓取
BV号为1jK411p7sa的视频属性抓取结束
开始BV号为1jV411k7i9视频属性的抓取
BV号为1jV411k7i9的视频属性抓取结束
开始BV号为1mz411i7bK视频属性的抓取
BV号为1mz411i7bK的视频属性抓取结束
开始BV号为16T4y1E73x视频属性的抓取
BV号为16T4y1E73x的视频属性抓取结束
开始BV号为18i4y1G7UR视频属性的抓取
BV号为18i4y1G7UR的视频属性抓取结束
开始BV号为1AZ4y1H7y1视频属性的抓取
BV号为1AZ4y1H7y1的视频属性抓取结束
开始BV号为175411W7gA视频属性的抓取
BV号为175411W7gA的视频属性抓取结束
开始BV号为1fk4y1z7TC视频属性的抓取
BV号为1fk4y1z7TC的视频属性抓取结束
开始BV号为1nv411673a视频属性的抓取
BV号为1nv411673a的视频属性抓取结束
开始BV号为1Eg4y1q7mU视频属性的抓取
BV号为1Eg4y1q7mU的视频属性抓取结束
开始BV号为1iT4y177Wn视频属性的抓取
BV号为1iT4y177Wn的视

开始BV号为1rv411v7R9视频属性的抓取
BV号为1rv411v7R9的视频属性抓取结束
开始BV号为13K411M7fX视频属性的抓取
BV号为13K411M7fX的视频属性抓取结束
开始BV号为1jK411T72f视频属性的抓取
BV号为1jK411T72f的视频属性抓取结束
开始BV号为1oA411J7xh视频属性的抓取
BV号为1oA411J7xh的视频属性抓取结束
开始BV号为1Gp4y1q7LT视频属性的抓取
BV号为1Gp4y1q7LT的视频属性抓取结束
开始BV号为1S54y1U7fr视频属性的抓取
BV号为1S54y1U7fr的视频属性抓取结束
开始BV号为1xV411z7BW视频属性的抓取
BV号为1xV411z7BW的视频属性抓取结束
开始BV号为1Fi4y1g7tr视频属性的抓取
BV号为1Fi4y1g7tr的视频属性抓取结束
开始BV号为15i4y1g7rS视频属性的抓取
BV号为15i4y1g7rS的视频属性抓取结束
开始BV号为1pZ4y1K7jw视频属性的抓取
BV号为1pZ4y1K7jw的视频属性抓取结束
开始BV号为1ph411Z7kF视频属性的抓取
BV号为1ph411Z7kF的视频属性抓取结束
开始BV号为1Dp4y1v7ZD视频属性的抓取
开始BV号为1Dp4y1v7ZD视频属性的抓取
BV号为1Dp4y1v7ZD的视频属性抓取结束
开始BV号为1mD4y1U7Gk视频属性的抓取
BV号为1mD4y1U7Gk的视频属性抓取结束
开始BV号为1S54y1U7RP视频属性的抓取
BV号为1S54y1U7RP的视频属性抓取结束
开始BV号为16Z4y1M7xC视频属性的抓取
BV号为16Z4y1M7xC的视频属性抓取结束
开始BV号为1ra4y1J7s8视频属性的抓取
BV号为1ra4y1J7s8的视频属性抓取结束
开始BV号为18C4y1t7dR视频属性的抓取
BV号为18C4y1t7dR的视频属性抓取结束
开始BV号为12Z4y1T77G视频属性的抓取
BV号为12Z4y1T77G的视频属性抓取结束
开始BV号为1jt4y1D7Yq视频属性的抓取
BV号为1jt4y1D7Yq的视频属性抓取结束
开始BV号为1zC4y1t7iz视频属性的抓取
BV号为1zC4y1t7iz的视频属性抓取结束
开始BV号为1xf4y197H7

开始BV号为1jt4y1i7UB视频属性的抓取
BV号为1jt4y1i7UB的视频属性抓取结束
开始BV号为1cr4y1w7Gu视频属性的抓取
BV号为1cr4y1w7Gu的视频属性抓取结束
开始BV号为1pi4y1j7yr视频属性的抓取
BV号为1pi4y1j7yr的视频属性抓取结束
开始BV号为1nz4y1o7Lw视频属性的抓取
BV号为1nz4y1o7Lw的视频属性抓取结束
开始BV号为1jD4y197qJ视频属性的抓取
BV号为1jD4y197qJ的视频属性抓取结束
开始BV号为1hf4y1B7no视频属性的抓取
BV号为1hf4y1B7no的视频属性抓取结束
开始BV号为1Ff4y1z7pY视频属性的抓取
BV号为1Ff4y1z7pY的视频属性抓取结束
开始BV号为1xZ4y1L7e8视频属性的抓取
BV号为1xZ4y1L7e8的视频属性抓取结束
开始BV号为1WT4y1w7wT视频属性的抓取
BV号为1WT4y1w7wT的视频属性抓取结束
开始BV号为1ct4y1e7as视频属性的抓取
BV号为1ct4y1e7as的视频属性抓取结束
开始BV号为1py4y1r7NV视频属性的抓取
BV号为1py4y1r7NV的视频属性抓取结束
开始BV号为1M5411L772视频属性的抓取
BV号为1M5411L772的视频属性抓取结束
开始BV号为1xz4y1o7Z3视频属性的抓取
BV号为1xz4y1o7Z3的视频属性抓取结束
开始BV号为1my4y1r7cW视频属性的抓取
BV号为1my4y1r7cW的视频属性抓取结束
开始BV号为1dr4y1c727视频属性的抓取
BV号为1dr4y1c727的视频属性抓取结束
开始BV号为14i4y1j7Bq视频属性的抓取
BV号为14i4y1j7Bq的视频属性抓取结束
开始BV号为1Cp4y1r72P视频属性的抓取
BV号为1Cp4y1r72P的视频属性抓取结束
开始BV号为1uD4y1975o视频属性的抓取
BV号为1uD4y1975o的视频属性抓取结束
开始BV号为1DK4y1f7TA视频属性的抓取
BV号为1DK4y1f7TA的视频属性抓取结束
开始BV号为1iD4y1d7dE视频属性的抓取
BV号为1iD4y1d7dE的视频属性抓取结束
开始BV号为1Ft4y1e7SM视频属性的抓取
BV号为1Ft4y1e7SM的视

In [14]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-14-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [12]:
channel_shuma_shouji.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [13]:
channel_shuma_shouji.to_json('channel_shuma_shouji.json')

In [17]:
pd.set_option('display.max_rows', None)

In [18]:
channel_youxi_danji[channel_youxi_danji['时间']=='4月']

,时间,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1h54y1d7Qq,4月,鹤吱菌,个人认证,191.3万,81,【师生对线】这网课上得就尼玛离谱！（第五集）,游戏,单机游戏,活动作品,自制,"[配音, 守望先锋, 独立游戏, 搞笑, 单机游戏, 游戏, 单机游戏, 电子竞技, 配音大...",13:00,953.7万,114.2万,115.2万,42.5万,8.9万,25302,18.8万,否
1L64y1M7Dp,4月,老番茄,个人认证,1360.1万,348,【老番茄】史上最骚骑士,游戏,单机游戏,活动作品,自制,"[骑马与砍杀, 搞笑, 游戏, 单机游戏, 老番茄, 骑马与砍杀2, 游戏练级挑战, 骑砍,...",15:45,902.3万,84.5万,61.7万,24.5万,2.4万,21336,9.7万,否
18K41157Tu,4月,老番茄,个人认证,1360.1万,348,宫斗大师老番茄,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 宫妃, 易次元]",13:55,739.6万,45.1万,9.3万,5.5万,1.3万,14189,10.6万,否
1PT4y1G7ME,4月,老番茄,个人认证,1360.1万,348,常胜将军老番茄,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 老番茄, shield wall]",14:54,726.8万,48.7万,12.9万,5.8万,7401,15180,7.0万,否
1S54y197uX,4月,徐大虾咯,个人认证,570.1万,224,家 里 有 矿 ！,游戏,单机游戏,普通视频,自制,"[我的世界, 单机游戏, 游戏, 单机游戏, 徐大虾咯, 徐大虾, MINECRAFT, M...",12:18,714.5万,35.0万,8.8万,4.3万,4860,7738,6.7万,否
1ti4y1t7YM,4月,老番茄,个人认证,1360.1万,348,解说挑战！什么游戏都能解说？,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 老番茄]",08:19,711.2万,61.4万,16.7万,8.1万,9970,14368,10.6万,否
1Ai4y1b7ET,4月,敖厂长,个人认证,735.1万,348,花了一万多元终于见到最早开放世界游戏之一,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 游戏资讯, 敖厂长, 单机, 游戏杂谈, 游戏视频, 杂谈]",14:26,670.4万,112.6万,156.8万,34.9万,2.4万,15618,8.6万,否
1Up4y1C7PB,4月,丢鹏D1uuu,无,8.4万,20,年级第一：别TM挡着我上网课！！！,游戏,单机游戏,活动作品,自制,"[独立游戏, 搞笑, 单机游戏, 游戏, 单机游戏, bilibili新星计划, 新人, 恶...",05:38,654.0万,17.3万,1.5万,2.0万,1321,1952,4526,否
1wi4y187bu,4月,中国BOY超级大猩猩,个人认证,663.5万,1380,全员铁憨憨!,游戏,单机游戏,活动作品,自制,"[STEAM, 单机游戏, 游戏, 单机游戏, 花少北, 老番茄, 中国BOY, LexBu...",19:57,653.2万,42.0万,23.9万,14.2万,2.5万,13493,9.7万,否
13t4y1U7kQ,4月,徐大虾咯,个人认证,570.1万,224,德华的世界#2,游戏,单机游戏,普通视频,自制,"[我的世界, 单机游戏, 游戏, 单机游戏, 徐大虾咯, 徐大虾, MINECRAFT, M...",12:07,619.7万,35.1万,9.5万,4.4万,7033,7937,6.3万,否


In [15]:
channel_shuma_shouji

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1w7411k7Ay,1月,2020-01-25 16:20:13,乔纳森何,无,571,7,87岁外公上手iPad写福字，89岁外婆把字挂在…,数码,手机平板,普通视频,...,"[iPhone, 数码, 手机平板, 数码, 平板, 苹果, 体验, IPADPRO, 春节...",00:15,130.6万,2.5万,1846,4874,1605,631,284,否
1f7411r7Pd,1月,2020-01-25 18:08:55,小圣嗷Sanche,无,6万,293,建议改为:轮 回 天 生 之 术,数码,手机平板,普通视频,...,"[垃圾佬, 手机, 数码, 手机平板, 数码, 游戏, 日常, 杂谈, 奸商, 闲鱼, 维修...",03:35,112.4万,3.5万,2959,4282,2158,2560,2714,否
1z7411B7w4,1月,2020-01-31 17:27:27,木羽说科技,个人认证,43万,90,6个我换手机舍不得删的小众 App——「木羽搞机14」,数码,手机平板,活动作品,...,"[手机, 数码, 手机平板, 数码, APP, 推荐, 全能打卡挑战]",03:38,100.6万,3.9万,2.2万,5.0万,1125,1980,2356,否
157411e7XB,1月,2020-01-22 15:58:50,木羽说科技,个人认证,43万,90,我的青春结束了——iPhone 11 深度体验（纯主观 不客观）「木羽体验报告第18期」,数码,手机平板,活动作品,...,"[手机, 数码, 手机平板, 数码, 我的装备清单2.0, 苹果手机, IPHONE, IO...",22:16,97.7万,5.2万,3.7万,1.2万,4094,4920,1.6万,否
147411z7sm,1月,2020-01-29 18:00:36,科技美学,个人认证,335.2万,1007,「科技美学」近期五款热门5G手机对比测评realme X50/Redmi K30/荣耀V30...,数码,手机平板,活动作品,...,"[5G, 科学, 数码, 手机平板, 数码, 跟我一起备年货, vivo X30 Pro, ...",22:42,94.3万,1.8万,4412,2286,3882,1777,7877,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1xy4y1C7Cp,10月,2020-10-19 00:18:16,肉包锅包肉,无,1644,223,华为Mate40,数码,手机平板,普通视频,...,"[数码, 手机平板, 数码, 华为, mate40]",10:01,36.1万,1446,24,124,230,566,1175,否
1MK4y1E7kS,10月,2020-10-22 20:29:29,戴表哥,无,1.4万,11,【戴表哥】华为Mate 40 Pro深度体验，史上最强的华为Mate,数码,手机平板,活动作品,...,"[手机评测, 手机, 数码, 手机平板, 数码, 数码玩家训练营, 华为, 测评, 安卓, ...",06:31,35.9万,1.1万,3285,1359,2682,1382,2098,否
1ji4y1j7ND,10月,2020-10-16 08:00:40,钟文泽,个人认证,133.9万,512,【小泽】索尼Xperia 1II体验：限定配色青山绿长这样！,数码,手机平板,活动作品,...,"[手机评测, 手机, 开箱, 数码, 手机平板, 数码, 数码嘉年华, 索尼, 安卓, SO...",13:33,34.5万,1.2万,2112,1246,2324,1439,4960,否
1ny4y1r7aS,10月,2020-10-30 17:00:28,华为终端,机构认证,98.3万,432,在一起，就可以,数码,手机平板,活动作品,...,"[黑科技, 数码, 手机平板, 数码, 数码玩家训练营, 华为, 华为Mate40, 华为2...",02:56,34.4万,3.6万,8253,7866,2952,934,767,否
